# Perspective-n-Point
Perspective-n-Point is the problem of estimating the pose of a **calibrated** camera given a set of n 3D points in the **world** and their correspondence projection point in the camera.
The estimated pose is thus the rotation (rvec) and the translation (tvec) vectors that allow transforming a 3D point **expressed in the world frame** into the **camera frame** $R_w^c, t_w^c$


<img src="images/pnp.jpg" alt="images/pnp.jpg" />

<img src="images/p3p.svg" alt="images/p3p.svg" />



First step: using cosine to relate the length in 4 triangles in the tetrahedron:


$s_1^2 = \frac{a^2}{u^2 + v^2 - 2uv \cos \alpha}$

$s_1^2 = \frac{b^2}{1 + v^2 - 2v \cos \beta}$


$s_1^2 = \frac{c^2}{1 + u^2 - 2u \cos \gamma}$


$A_4 v^4 + A_3 v^3 + A_2 v^2 + A_1 v + A_0 = 0$


Second step:


$
A_0 = \left(1 + \frac{a^2}{b^2} - \frac{c^2}{b^2}\right)^2 - \frac{4a^2}{b^2} \cos^2 \gamma
$

$
A_1 = 4 \left[ 
    - \left( \frac{a^2 - c^2}{b^2} \right) \left( 1 + \frac{a^2 - c^2}{b^2} \right) \cos \beta 
    + \frac{2a^2}{b^2} \cos^2 \gamma \cos \beta 
    - \left( 1 - \frac{a^2 + c^2}{b^2} \right) \cos \alpha \cos \gamma 
\right]
$

$
A_2 = 2 \left[ 
    \left( \frac{a^2 - c^2}{b^2} \right)^2 - 1 + 2 \left( \frac{a^2 - c^2}{b^2} \right)^2 \cos^2 \beta 
    + 2 \left( \frac{b^2 - c^2}{b^2} \right) \cos^2 \alpha 
    - 4 \left( \frac{a^2 + c^2}{b^2} \right) \cos \alpha \cos \beta \cos \gamma 
    + 2 \left( \frac{b^2 - a^2}{b^2} \right) \cos^2 \gamma 
\right]
$

$
A_3 = 4 \left[ 
    \frac{a^2 - c^2}{b^2} \left( 1 - \frac{a^2 - c^2}{b^2} \cos \beta \right) 
    - \left( 1 - \frac{a^2 + c^2}{b^2} \right) \cos \alpha \cos \gamma 
    + 2 \frac{c^2}{b^2} \cos^2 \alpha \cos \beta 
\right]
$

$
A_4 = \left( \frac{a^2 - c^2}{b^2} - 1 \right)^2 - \frac{4c^2}{b^2} \cos^2 \alpha
$

Which gives us for possible solution, so we need a 4th point or initial guess. The solution looks like these 4 tetrahedrons:


<img src="images/p3p_results.png" alt="images/p3p_results.png" />



## Critical Cylinder

<img src="images/critical_cylinder.png" alt="images/critical_cylinder.png" />

The solution gets in-stable

Refs: [1](https://www.youtube.com/watch?v=N1aCvzFll6Q), [2](https://www.cis.upenn.edu/~cis580/Spring2015/Lectures/cis580-13-LeastSq-PnP.pdf), [3](https://www.youtube.com/watch?v=xdlLXEyCoJY)

## P3P
```cpp
solveP3P	(	InputArray 	objectPoints,
InputArray 	imagePoints,
InputArray 	cameraMatrix,
InputArray 	distCoeffs,
OutputArrayOfArrays 	rvecs,
OutputArrayOfArrays 	tvecs,
int 	flags 
)	
```

The cv::solveP3P() computes an object pose from exactly 3 3D-2D point correspondences. A P3P problem has up to 4 solutions. The solutions are **sorted by reprojection errors** (lowest to highest).


```cpp

std::vector<cv::Vec3f> objectPointsInWorldCoordinate = {{-0.5, 0.5, 1}, {-0.2, 0.8, 0.5}, {0.8, 0, 0.7}};

std::vector<cv::Mat> tvecs, rvecs;

int flags = cv::SOLVEPNP_P3P; // or SOLVEPNP_AP3P
cv::solveP3P(objectPointsInWorldCoordinate, imagePointsCamera, cameraMatrix,
               cv::noArray(), rvecs, tvecs, flags);
```               


<img src="images/p3p_rerun.png" alt="images/p3p_rerun.png" width="50%" height="50%" />

[source](../src/src/P3P.cpp)



# PnP
The **Perspective-n-Point (PnP)** problem involves estimating the  $ R_w^c$, $t_w^c$ of a calibrated camera given a set of $ n $ 3D points in the **world frame** and their corresponding 2D projections in the image.

### Problem Definition

Given:
- **3D points in the world frame**: $ \mathbf{X}_i = (X_i, Y_i, Z_i)^\top $
- **2D points in the image frame**: $ \mathbf{x}_i = (u_i, v_i)^\top $
- **Camera intrinsic matrix** $ K $:

 
  $
  K = \begin{bmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{bmatrix}
  $


The goal is to estimate the camera's extrinsic parameters ( $ R_w^c$, $t_w^c$ ) such that the 3D points project correctly to their 2D counterparts.

The camera projection model is:

$
\mathbf{x}_i = K \cdot \left( R_w^c \cdot \mathbf{X}_i^w + t_w^c \right)
$

In homogeneous coordinates:

$
s \begin{bmatrix} u_i \\ v_i \\ 1 \end{bmatrix} = K \begin{bmatrix} R & t \end{bmatrix} \begin{bmatrix} X_i \\ Y_i \\ Z_i \\ 1 \end{bmatrix}
$

Where $ s $ is a scalar (the depth of the point in the camera frame).

---


### Expanding the Projection Equation

Let's expand the homogeneous projection step-by-step:

1. **Project to the camera frame**:
   $
   \mathbf{X}_{\text{camera}} = R \cdot \mathbf{X}_i + t
   $
   Where:
   - $ R $ is a $ 3 \times 3 $ matrix.
   - $ t $ is a $ 3 \times 1 $ vector.

2. **Project to the image plane**:
   $
   s \begin{bmatrix} u_i \\ v_i \\ 1 \end{bmatrix} = K \cdot \mathbf{X}_{\text{camera}}
   $

   Substituting $ \mathbf{X}_{\text{camera}} $:
   $
   s \begin{bmatrix} u_i \\ v_i \\ 1 \end{bmatrix} = K \cdot \left( R \cdot \mathbf{X}_i + t \right)
   $

3. **Decompose and rearrange**:
   For each point $ \mathbf{X}_i = (X_i, Y_i, Z_i)^\top $, the expanded equation is:

   $
   s \begin{bmatrix} u_i \\ v_i \\ 1 \end{bmatrix} = 
   \begin{bmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{bmatrix}
   \left( 
   \begin{bmatrix} r_{11} & r_{12} & r_{13} \\ r_{21} & r_{22} & r_{23} \\ r_{31} & r_{32} & r_{33} \end{bmatrix}
   \begin{bmatrix} X_i \\ Y_i \\ Z_i \end{bmatrix} +
   \begin{bmatrix} t_x \\ t_y \\ t_z \end{bmatrix}
   \right)
   $

Where:
- $ r_{ij} $ are the elements of $ R $.
- $ t_x, t_y, t_z $ are the elements of $ t $.

---


### Formulating $ \mathbf{A} \cdot \mathbf{p} = 0 $

To solve this linearly, we eliminate $ s $ by dividing the equation by the depth ($ Z_{\text{camera}} $).

1. The projection equations in terms of $ R $ and $ t $ are:
 
   $
   u_i = f_x \frac{(r_{11} X_i + r_{12} Y_i + r_{13} Z_i + t_x)}{(r_{31} X_i + r_{32} Y_i + r_{33} Z_i + t_z)} + c_x
   $
   
   $
   v_i = f_y \frac{(r_{21} X_i + r_{22} Y_i + r_{23} Z_i + t_y)}{(r_{31} X_i + r_{32} Y_i + r_{33} Z_i + t_z)} + c_y
   $

3. Rearrange these into a linear system:
 
   $
   u_i (r_{31} X_i + r_{32} Y_i + r_{33} Z_i + t_z) = f_x (r_{11} X_i + r_{12} Y_i + r_{13} Z_i + t_x) + c_x (r_{31} X_i + r_{32} Y_i + r_{33} Z_i + t_z)
   $
   

   $
   v_i (r_{31} X_i + r_{32} Y_i + r_{33} Z_i + t_z) = f_y (r_{21} X_i + r_{22} Y_i + r_{23} Z_i + t_y) + c_y (r_{31} X_i + r_{32} Y_i + r_{33} Z_i + t_z)
   $

4. Rewrite in matrix form $ \mathbf{A} \cdot \mathbf{p} = 0 $, where:
   - $ \mathbf{A} $ is constructed from the 2D and 3D correspondences.
   - $ \mathbf{p} $ contains the unknowns $ r_{11}, r_{12}, \ldots, t_x, t_y, t_z $.

---

### Example of $ \mathbf{A} $

For one point $ \mathbf{X}_i $:
$
\mathbf{A} =
\begin{bmatrix}
X_i & Y_i & Z_i & 1 & 0 & 0 & 0 & 0 & -u_i X_i & -u_i Y_i & -u_i Z_i & -u_i \\
0 & 0 & 0 & 0 & X_i & Y_i & Z_i & 1 & -v_i X_i & -v_i Y_i & -v_i Z_i & -v_i
\end{bmatrix}
$

For $ n $ points, stack the rows for all points to form $ \mathbf{A} $.

---

### Solving Using SVD

To solve $ \mathbf{A} \cdot \mathbf{p} = 0 $:
1. Perform SVD on $ \mathbf{A} $:
   $
   \mathbf{A} = \mathbf{U} \cdot \Sigma \cdot \mathbf{V}^\top
   $
2. The solution $ \mathbf{p} $ is the last column of $ \mathbf{V} $, corresponding to the smallest singular value.

3. Reshape $ \mathbf{p} $ into $ R $ and $ t $.

---

This method provides an initial estimate of $ R $ and $ t $, often refined using non-linear optimization techniques like Levenberg-Marquardt.


### Notes:

1. **Constraints on $ R $**:
   - The rotation matrix $ R $ must be orthogonal ($ R^\top R = I $) with determinant $ +1 $. After solving, $ R $ may need re-orthogonalization (e.g., using the Procrustes method or SVD).

2. **Handling Noise**:
   - Real-world data introduces noise, so iterative methods like Levenberg-Marquardt are often applied to refine the solution after obtaining an initial estimate using SVD.

3. **Minimal Points**:
   - A minimal number of $ n = 4 $ points is needed to solve the PnP problem linearly. For robustness, $ n > 4 $ is preferred.


## Pose computation methods
### 1. cv::solvePnP()
The `cv::solvePnP()` returns the rotation and the translation vectors that transform a 3D point expressed in the **object coordinate frame** to the **camera coordinate frame**, $ R_w^c$, $t_w^c$ 


```cpp
bool cv::solvePnP	(	InputArray 	objectPoints,
InputArray 	imagePoints,
InputArray 	cameraMatrix,
InputArray 	distCoeffs,
OutputArray 	rvec,
OutputArray 	tvec,
bool 	useExtrinsicGuess = false,
int 	flags = SOLVEPNP_ITERATIVE 
)	
```

The flags `cv::SolvePnPMethod` enum values:

1. `SOLVEPNP_P3P`, `SOLVEPNP_AP3P`:  requires **exactly** 4 input points to return a unique solution.

2. `cv::SOLVEPNP_IPPE`: Input points must be >= 4 and object points must be **coplanar**. 

3. `cv::SOLVEPNP_IPPE_SQUARE`: special case suitable for marker pose estimation. Number of input points must be 4. Object points must be defined in the following order: 

    - point 0: [-squareLength / 2, squareLength / 2, 0]
    - point 1: [ squareLength / 2,squareLength / 2, 0]
    - point 2: [ squareLength / 2, -squareLength / 2, 0]
    - point 3: [-squareLength / 2, -squareLength / 2, 0]

4. `SOLVEPNP_ITERATIVE`:  based on a Levenberg-Marquardt optimization, minimizes reprojection error, Initial solution for non-planar "objectPoints" needs at least **6 points** and uses the **DLT algorithm**. Initial solution for planar "objectPoints" needs at least **4 points** and uses pose from homography decomposition. `useExtrinsicGuess`:	Parameter used for `SOLVEPNP_ITERATIVE`. If true (1), the function uses the provided `rvec` and `tvec` values as initial With SOLVEPNP_ITERATIVE method and useExtrinsicGuess=true, the minimum number of points is 3 (3 points are sufficient to compute a pose but there are up to 4 solutions). The initial solution should be close to the global solution to converge.


5. `cv::SOLVEPNP_EPNP`: Efficient Perspective-n-Point Camera Pose Estimation
6. `cv::SOLVEPNP_IPPE`: This method requires **coplanar** object points.
7. `cv::SOLVEPNP_SQPNP`: It requires **3** or more points.


### 2. cv::solvePnPGeneric() 
The `cv::solvePnPGeneric()` allows retrieving all the possible solutions.
###  3. cv::solvePnPRansac() 
The `cv::solvePnPRansac()` computes the object pose wrt. the camera frame using a RANSAC scheme to deal with outliers.

## Pose refinement
Pose refinement consists in estimating the rotation and translation that minimizes the reprojection. `cv::solvePnPRefineLM()` and `cv::solvePnPRefineVVS()`

Refs: [1](https://docs.opencv.org/4.x/d5/d1f/calib3d_solvePnP.html), [2](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga549c2075fac14829ff4a58bc931c033d)

<img src="images/Perspective-n-Point.svg" alt="images/Perspective-n-Point.svg" width="90%" height="90%" />

```bash
object_points:
 [[ 0.  0.  0.]
 [10.  0.  0.]
 [10. 10.  0.]
 [ 0. 10.  0.]]
rotation of object in camera: 
 [[0.813104  ]
 [0.00000002]
 [3.03454547]] 
 translation of object in camera:
 [[-20.00000019]
 [ -0.00000006]
 [ 50.00000046]]
```




[code](../script/perspective-n-point.py)